In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmpux5o8hqk
  JVM stdout: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmpux5o8hqk/h2o_julio_started_from_python.out
  JVM stderr: /var/folders/h0/4hznbs6x77gf8d5h9lp1hkwr0000gn/T/tmpux5o8hqk/h2o_julio_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_julio_bty8a7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.778 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
import pandas as pd

In [32]:
diamonds=pd.read_csv('diamonds_train.csv', index_col=0)
diamonds=h2o.H2OFrame(diamonds)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
diamonds=diamonds.asnumeric()

In [34]:
diamonds.columns

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price']

In [35]:
y=["price"]
X=['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']

In [36]:
train,test=diamonds.split_frame(ratios=[0.8])

In [37]:
X_train=train[X]
y_trin=train[y]
X_test=test[X]
y_test=test[y]

In [38]:
from h2o.automl import H2OAutoML

In [42]:
aml_ti=H2OAutoML(max_runtime_secs=180,max_models=40,seed=1,nfolds=0)
aml_ti.train(x=x_columns,y=y_columns,training_frame=train,validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
lb_ti=aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_4_AutoML_20200511_181550,286385,535.149,286385,270.623,0.094061
GBM_grid__1_AutoML_20200511_181550_model_1,294933,543.077,294933,278.528,0.10086
GBM_3_AutoML_20200511_181550,296348,544.379,296348,278.896,0.101393
GBM_2_AutoML_20200511_181550,297693,545.612,297693,282.238,0.105572
GBM_1_AutoML_20200511_181550,298913,546.729,298913,282.314,0.104606
GBM_5_AutoML_20200511_181550,315527,561.718,315527,282.58,0.101645
XGBoost_grid__1_AutoML_20200511_181550_model_6,320318,565.967,320318,286.722,0.0962034
XGBoost_grid__1_AutoML_20200511_181550_model_2,320813,566.404,320813,287.749,0.0960001
GBM_grid__1_AutoML_20200511_181550_model_6,326831,571.692,326831,305.475,0.126131
XGBoost_3_AutoML_20200511_181550,331371,575.648,331371,302.648,0.115979


In [ ]:
y_pred=aml_ti.leader.predict(X_test)

In [12]:
test = pd.read_csv('diamonds_test.csv', index_col=0)
test=h2o.H2OFrame(test)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
test=test.asnumeric()

In [15]:
predictions=aml_ti.leader.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [16]:
preds2=preds.as_data_frame(use_pandas=True)

In [17]:
preds2

,predict
0,430.867200
1,1365.027615
2,9527.580114
3,533.300843
4,9050.025180
...,...
13444,4875.454269
13445,462.268462
13446,15190.629623
13447,10724.353270


In [19]:
test=test.as_data_frame(use_pandas=True)

In [20]:
test

,carat,cut,color,clarity,depth,table,x,y,z
0,0.30,2,4,3,60.0,56.0,4.41,4.43,2.65
1,0.34,2,0,1,62.1,57.0,4.52,4.46,2.79
2,1.57,4,5,5,60.3,58.0,7.58,7.55,4.56
3,0.31,2,4,5,61.8,57.0,4.32,4.36,2.68
4,1.51,1,5,6,64.0,60.0,7.26,7.21,4.63
...,...,...,...,...,...,...,...,...,...
13444,1.05,2,2,3,60.3,55.0,6.55,6.68,3.99
13445,0.27,4,3,4,61.9,56.0,4.13,4.16,2.56
13446,1.50,3,0,5,61.8,60.0,7.30,7.37,4.53
13447,1.70,3,2,3,60.2,61.0,7.79,7.68,4.66


In [28]:
result=pd.DataFrame(preds2).reset_index()

In [30]:
result

,index,predict
0,0,430.867200
1,1,1365.027615
2,2,9527.580114
3,3,533.300843
4,4,9050.025180
...,...,...
13444,13444,4875.454269
13445,13445,462.268462
13446,13446,15190.629623
13447,13447,10724.353270


In [43]:
results.to_csv("my_submition18",index=False)